<a href="https://colab.research.google.com/github/shashi3876/ADIA_Lab_Structural_Break_Challenge/blob/main/Fifth_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/badge/open-in-colab.svg)](https://colab.research.google.com/github/crunchdao/quickstarters/blob/master/competitions/structural-break/quickstarters/random-submission/random-submission.ipynb)

![Banner](https://raw.githubusercontent.com/crunchdao/quickstarters/refs/heads/master/competitions/structural-break/assets/banner.webp)

# Setup

The first steps to get started are:
1. Get the setup command
2. Execute it in the cell below

### >> https://hub.crunchdao.com/competitions/structural-break/submit/notebook

![Reveal token](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/animations/reveal-token.gif)

In [1]:
# Install the Crunch CLI
%pip install --upgrade crunch-cli

# Setup your local environment
!crunch setup --notebook structural-break hello --token w7U0Bc87URhfqG8IL78lerPk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.4/118.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: requirements-parser
    Found existing installation: requirements-parser 0.9.0
    Uninstalling requirements-parser-0.9.0:
      Successfully uninstalled requirements-parser-0.9.0
crunch-cli, version 7.4.0
main.py: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/submissions/26430/main.py (8465 bytes)
notebook.ipynb: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/submissions/26430/notebook.ipynb (83391 bytes)
requirements.txt: downlo

# Your model

## Setup

In [2]:
import os
import random
import typing

# Import your dependencies
import joblib
import pandas as pd
import sklearn.metrics

In [3]:
import crunch

# Load the Crunch Toolings
crunch = crunch.load_notebook()

loaded inline runner with module: <module '__main__'>

cli version: 7.4.0
available ram: 12.67 gb
available cpu: 2 core
----


## Data

The data was downloaded when you setup your local environment and is now available in the `data/` directory.

In [4]:
# Load the data simply
X_train, y_train, X_test = crunch.load_data()

data/X_train.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/X_train.parquet (204327238 bytes)
data/X_train.parquet: already exists, file length match
data/X_test.reduced.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/X_test.reduced.parquet (2380918 bytes)
data/X_test.reduced.parquet: already exists, file length match
data/y_train.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/y_train.parquet (61003 bytes)
data/y_train.parquet: already exists, file length match
data/y_test.reduced.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/y_test.reduced.parquet (2655 bytes)
data/y_test.reduced.parquet: already exists, file length match


### `X_train`

Index:
- `id`: the ID of the dataset
- `time`: arbitrary amount of time sampled regularely

Columns:
- `value`: the timeseries data
- `period`: if you are in an **initial segment** (0) or an **extension segment** (1)

In [5]:
X_train

value  period
id    time                  
0     0    -0.005564       0
      1     0.003705       0
      2     0.013164       0
      3     0.007151       0
      4    -0.009979       0
...              ...     ...
10000 2134  0.001137       1
      2135  0.003526       1
      2136  0.000687       1
      2137  0.001640       1
      2138  0.001074       1

[23715734 rows x 2 columns]

### `y_train`

This is a simple `pandas.Series` that tells if a dataset id has a structural breakpoint or not.

Index:
- `id`: the ID of the dataset

Value:
- `structural_breakpoint`: the value you need to predict

In [6]:
y_train

,structural_breakpoint
id,
0,False
1,False
2,True
3,False
4,False
...,...
9996,False
9997,False
9998,False


### `X_test`

This is a **`list` of `pandas.DataFrame`** that have the same format as [`X_train`](#X_train).

It is provided as a list to make sure you are encouraged to read the records **one by one**, __as this will be mandatory in the [`infer()`](#infer) function__.

In [7]:
print("Number of datasets:", len(X_test))

Number of datasets: 101


In [10]:
import pandas as pd
import numpy as np
import os
import joblib
import time
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, roc_auc_score
from scipy.stats import ks_2samp, mannwhitneyu, ttest_ind, wasserstein_distance

try:
    from xgboost import XGBClassifier
    xgb_available = True
except ImportError:
    xgb_available = False

try:
    from lightgbm import LGBMClassifier
    lgbm_available = True
except ImportError:
    lgbm_available = False


# --- Statistical functions ---
def cohen_d(x, y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    pooled_std = np.sqrt(((nx - 1) * np.std(x, ddof=1) ** 2 +
                          (ny - 1) * np.std(y, ddof=1) ** 2) / dof)
    return (np.mean(x) - np.mean(y)) / pooled_std if pooled_std != 0 else 0


def cliffs_delta(x, y):
    x = np.array(x)
    y = np.array(y)
    more = sum(xi > yj for xi in x for yj in y)
    less = sum(xi < yj for xi in x for yj in y)
    return (more - less) / (len(x) * len(y))


def train(X_train: pd.DataFrame, y_train: pd.Series, model_directory_path: str):
    # Pre-split data
    pre_groups = X_train.loc[X_train["period"] == 0].groupby("id")["value"].apply(np.array)
    post_groups = X_train.loc[X_train["period"] == 1].groupby("id")["value"].apply(np.array)

    ids = sorted(set(pre_groups.index) & set(post_groups.index))
    features_df = pd.DataFrame({"id": ids})

    def apply_stat(func, name, fill_value=np.nan):
        start_time = time.time()
        results = []
        for id_val in ids:
            try:
                results.append(func(pre_groups[id_val], post_groups[id_val]))
            except Exception:
                results.append(fill_value)
        features_df[name] = results
        elapsed = time.time() - start_time
        print(f"{name} computed in {elapsed:.3f} seconds")

    # KS test
    apply_stat(lambda pre, post: ks_2samp(pre, post)[0], "ks_stat")
    apply_stat(lambda pre, post: ks_2samp(pre, post)[1], "ks_p")

    # Wasserstein distance
    apply_stat(lambda pre, post: wasserstein_distance(pre, post), "wasserstein")

    # Mann-Whitney U test
    apply_stat(lambda pre, post: mannwhitneyu(pre, post, alternative="two-sided")[0], "mannwhitney_stat")
    apply_stat(lambda pre, post: mannwhitneyu(pre, post, alternative="two-sided")[1], "mannwhitney_p")

    # Cohen's d
    apply_stat(lambda pre, post: cohen_d(pre, post), "cohen_d")

    # T-test
    apply_stat(lambda pre, post: ttest_ind(pre, post, equal_var=False)[0], "ttest_stat")
    apply_stat(lambda pre, post: ttest_ind(pre, post, equal_var=False)[1], "ttest_p")

    print("All features calculated.")

    # Add target
    features_df["target"] = y_train.loc[features_df["id"]].values.astype(bool)

    # Train-test split
    X = features_df.drop(columns=["id", "target"])
    y = features_df["target"]
    # Ensure y is int 0/1
    y = features_df["target"].astype(int)

    # Use built-in scorer
    scorer = "roc_auc"
    # --- Model definitions & parameter grids ---
    models_and_params = {
        "RandomForest": (
            RandomForestClassifier(random_state=42),
            {
                "n_estimators": [100, 200, 300],
                "max_depth": [None, 5, 10],
                "min_samples_split": [2, 5, 10]
            }
        ),
        "GradientBoosting": (
            GradientBoostingClassifier(random_state=42),
            {
                "n_estimators": [100, 200],
                "learning_rate": [0.05, 0.1, 0.2],
                "max_depth": [3, 5]
            }
        ),
        "LogisticRegression": (
            LogisticRegression(max_iter=500, solver='liblinear', random_state=42),
            {
                "C": [0.1, 1, 10]
            }
        )
    }

    if xgb_available:
        models_and_params["XGBoost"] = (
            XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
            {
                "n_estimators": [100, 200],
                "learning_rate": [0.05, 0.1, 0.2],
                "max_depth": [3, 5]
            }
        )

    if lgbm_available:
        models_and_params["LightGBM"] = (
            LGBMClassifier(random_state=42),
            {
                "n_estimators": [100, 200],
                "learning_rate": [0.05, 0.1, 0.2],
                "max_depth": [-1, 5, 10]
            }
        )

    # --- Cross-validation setup ---
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    #scorer = make_scorer(roc_auc_score, needs_proba=True)

    best_model = None
    best_score = -np.inf
    best_name = None

    # --- Model search ---
    for name, (model, param_grid) in models_and_params.items():
        print(f"Searching best params for {name}...")
        grid = GridSearchCV(model, param_grid, scoring=scorer, cv=cv, n_jobs=-1)
        grid.fit(X, y)
        print(f"{name} best CV ROC AUC: {grid.best_score_:.4f} with params {grid.best_params_}")

        if grid.best_score_ > best_score:
            best_score = grid.best_score_
            best_model = grid.best_estimator_
            best_name = name

    print(f"Selected best model: {best_name} with CV ROC AUC: {best_score:.4f}")

    # --- Fit best model on all data ---
    best_model.fit(X, y)

    # Save the best model
    os.makedirs(model_directory_path, exist_ok=True)
    joblib.dump(best_model, os.path.join(model_directory_path, "model.joblib"))
    print(f"Best model saved to {model_directory_path}")

## Implementation

### `train()`

In the training function, users build and train the model to make inferences on the test data. <br />
Your model must be stored in the `model_directory_path`.

In [11]:
train(X_train,y_train,'/')

ks_stat computed in 20.513 seconds
ks_p computed in 20.629 seconds
wasserstein computed in 3.827 seconds
mannwhitney_stat computed in 13.329 seconds
mannwhitney_p computed in 12.276 seconds
cohen_d computed in 0.777 seconds
ttest_stat computed in 11.542 seconds
ttest_p computed in 11.655 seconds
All features calculated.
Searching best params for RandomForest...
RandomForest best CV ROC AUC: 0.6180 with params {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 100}
Searching best params for GradientBoosting...
GradientBoosting best CV ROC AUC: 0.6113 with params {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}
Searching best params for LogisticRegression...
LogisticRegression best CV ROC AUC: 0.5021 with params {'C': 0.1}
Searching best params for XGBoost...


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [20:59:41] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost best CV ROC AUC: 0.6127 with params {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}
Searching best params for LightGBM...
[LightGBM] [Info] Number of positive: 2909, number of negative: 7092
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000918 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 10001, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.290871 -> initscore=-0.891158
[LightGBM] [Info] Start training from score -0.891158
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

### `infer()`

In the inference function, the trained model is loaded and used to make inferences on a sample of data that matches the characteristics of the training test.

#### Setup

Once your model is loaded, you must do a `yield` to signal it to the runner. <br />
After that you can start reading data from `X_test`.

#### Iteration

The datasets must be read **one by one** and each value must be returned with a `yield <value>`. <br />
If you try to skip this, you will get an error. <br />
All values are then concatenated into a prediction file.

**Warning: The datasets can only be iterated once!**

#### Cleanup

Code can be executed after the `for` loop if you need to persist state or do some cleanup.

In [12]:
def infer(
    X_test: typing.Iterable[pd.DataFrame],
    model_directory_path: str,
) -> typing.Generator[bool, None, None]:
    """
    Generator that yields predictions for each sample in X_test.
    Each sample is a DataFrame with columns ['id', 'period', 'value'].
    """

    model = joblib.load(os.path.join(model_directory_path, "model.joblib"))

    yield  # Mark as ready for streaming

    for dataset in X_test:
        # Ensure dataset has both pre and post
        pre_values = dataset.loc[dataset["period"] == 0, "value"].values
        post_values = dataset.loc[dataset["period"] == 1, "value"].values

        if len(pre_values) == 0 or len(post_values) == 0:
            yield False  # or np.nan depending on desired behavior
            continue

        feats = {}

        # KS test
        ks_stat, ks_p = ks_2samp(pre_values, post_values)
        feats["ks_stat"] = ks_stat
        feats["ks_p"] = ks_p

        # Wasserstein
        feats["wasserstein"] = wasserstein_distance(pre_values, post_values)

        # Mann-Whitney
        try:
            mw_stat, mw_p = mannwhitneyu(pre_values, post_values, alternative="two-sided")
        except ValueError:
            mw_stat, mw_p = np.nan, np.nan
        feats["mannwhitney_stat"] = mw_stat
        feats["mannwhitney_p"] = mw_p

        # Cohen's d
        feats["cohen_d"] = cohen_d(pre_values, post_values)

        # Cliff's delta (optional — uncomment if needed)
        # feats["cliffs_delta"] = cliffs_delta(pre_values, post_values)

        # T-test
        t_stat, t_p = ttest_ind(pre_values, post_values, equal_var=False)
        feats["ttest_stat"] = t_stat
        feats["ttest_p"] = t_p

        # Create DF for model
        X_features = pd.DataFrame([feats])

        prediction = model.predict(X_features)[0]
        yield bool(prediction)

## Local testing

To make sure your `train()` and `infer()` function are working properly, you can call the `crunch.test()` function that will reproduce the cloud environment locally. <br />
Even if it is not perfect, it should give you a quick idea if your model is working properly.

In [ ]:
crunch.test(
    # Uncomment to disable the train
    # force_first_train=False,

    # Uncomment to disable the determinism check
    # no_determinism_check=True,
)

21:01:16 no forbidden library found
21:01:16 
21:01:16 started
21:01:16 running local test
21:01:16 internet access isn't restricted, no check will be done
21:01:16 
21:01:17 starting unstructured loop...
21:01:17 executing - command=train


data/X_train.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/X_train.parquet (204327238 bytes)
data/X_train.parquet: already exists, file length match
data/X_test.reduced.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/X_test.reduced.parquet (2380918 bytes)
data/X_test.reduced.parquet: already exists, file length match
data/y_train.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/y_train.parquet (61003 bytes)
data/y_train.parquet: already exists, file length match
data/y_test.reduced.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/y_test.reduced.parquet (2655 bytes)
data/y_test.reduced.parquet: already exists, file length match
ks_stat computed in 20.338 seconds
ks_p computed in 20.341 seconds
wasserstein computed in 4.302 seconds
mannwhitney_

## Results

Once the local tester is done, you can preview the result stored in `data/prediction.parquet`.

In [ ]:
prediction = pd.read_parquet("data/prediction.parquet")
prediction

,prediction
id,
10001,0.0
10002,0.0
10003,0.0
10004,0.0
10005,0.0
...,...
10097,0.0
10098,0.0
10099,0.0


### Local scoring

You can call the function that the system uses to estimate your score locally.

In [ ]:
# Load the targets
target = pd.read_parquet("data/y_test.reduced.parquet")["structural_breakpoint"].astype(float)

# Call the scoring function
sklearn.metrics.roc_auc_score(
    target,
    prediction,
)

np.float64(0.5551643192488263)

# Submit your Notebook

To submit your work, you must:
1. Download your Notebook from Colab
2. Upload it to the platform
3. Create a run to validate it

### >> https://hub.crunchdao.com/competitions/structural-break/submit/notebook

![Download and Submit Notebook](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/animations/download-and-submit-notebook.gif)